Using the Scikit-Learn framework, build a set of functioning binary classifiers using provided training data. Use comments to document your major steps (what and why) and any important decisions you make along the way. You can choose any performance metric(s) you like but please include AUC score as an output for the grader.



To access the training data, clone this public GitHub repo and unzip the file called 'testing_data.csv'

https://github.com/MarletteFunding/marlette-ds-challenge



The data columns beginning with 'var' are numeric, and 'cat' are categorical. The target column is labeled 'target'.



Requirements
1. Build (3) Binary Classifiers using Logistic, Catboost, XGBoost algorithms. Pick the best & measure the metrics (accuracy, precision & recall, AUC score) using the provided training data. You may ensemble them if you wish. 

2. Modify the objective & evaluation functions of algorithm of choice to beat the precision of the model in the first requirement

3. Develop code to find a sample weighting scheme that produces better precision than the model in the second requirement

4. Document the key steps in your workflow (what and why)

5. Your code should be replicable so a grader can run your code and achieve the same results



How to submit
Please upload the code for this project to GitHub, and post a link to your repository below. Make sure it is publicly accessible.

# Imports

In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_fscore_support, plot_confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


# set a seed for reproducibility
RANDOM_STATE = 99

# Feature engineering

In [3]:
df = pd.read_csv('training_data.csv')

In [4]:
df.head()

,ID,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var189,var190,var191,var192,cat1,cat2,cat3,cat4,cat5,target
0,44686,86.52893,80.79771,75.25887,74.02016,69.01476,65.61648,63.23896,59.07834,56.80397,...,85.133333,84.45,85.20,85.900000,S,H,C,B,C,0
1,44687,68.56225,72.05599,69.52573,68.79211,65.48515,63.00976,61.19186,57.85757,55.94791,...,90.533333,86.55,87.24,87.300000,S,I,C,B,C,0
2,44688,77.88821,76.62270,73.11046,72.20956,68.26166,65.34046,63.19467,59.25676,57.01834,...,93.933333,90.20,89.84,88.600000,S,I,C,B,C,0
3,44689,81.11949,78.43038,74.59578,73.63714,69.45540,66.35951,64.07976,59.88543,57.50303,...,93.200000,88.15,88.48,87.766667,S,I,C,B,C,0
4,44690,62.18698,68.60618,67.86709,67.44987,65.15601,63.13671,61.52867,58.35072,56.42460,...,92.733333,88.15,88.00,88.566667,S,I,C,B,C,0


In [5]:
df.describe(include='all')

,ID,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var189,var190,var191,var192,cat1,cat2,cat3,cat4,cat5,target
count,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,14193.000000,...,14193.000000,14193.000000,14193.000000,14193.000000,14193,14193,14193,14193,14193,14193.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20,9,3,3,3,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,M,H,B,B,B,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,716,1577,7091,12814,11367,NaN
mean,60691.679067,49.770732,49.816368,49.842512,49.847619,49.866617,49.877695,49.883878,49.889778,49.889927,...,64.371634,64.739135,65.117889,65.514350,NaN,NaN,NaN,NaN,NaN,0.009441
std,9537.840350,19.287088,13.258854,11.027864,10.617691,9.063626,8.015570,7.248920,5.796510,4.958480,...,51.892179,51.490296,51.202395,51.035665,NaN,NaN,NaN,NaN,NaN,0.096710
min,44686.000000,1.962090,6.802490,10.049460,10.790220,14.121840,16.951820,19.406260,25.208090,29.421340,...,1.666667,2.050000,2.160000,2.066667,NaN,NaN,NaN,NaN,NaN,0.000000
25%,52284.000000,35.131290,40.745080,42.377800,42.688460,43.737880,44.455170,44.995210,46.016590,46.572020,...,22.533333,22.950000,22.880000,23.400000,NaN,NaN,NaN,NaN,NaN,0.000000
50%,60726.000000,49.798510,49.924220,50.051180,50.093620,50.080180,50.101920,50.063300,49.967990,49.864340,...,50.866667,51.050000,51.400000,51.266667,NaN,NaN,NaN,NaN,NaN,0.000000
75%,69174.000000,64.213330,59.015310,57.290670,57.032030,55.972370,55.194490,54.661910,53.790120,53.208110,...,92.066667,92.900000,93.760000,94.433333,NaN,NaN,NaN,NaN,NaN,0.000000


The describe method on the 5 categorical variables shows that the highest number of categories is 20 for cat1.  Encoding the 5 categorical variables will result in 33 columns (since we can drop one category for every variable) compared to the original 5.  Given that there are so many numerical variables, one hot encoding will not result in an overly sparse matrix, so this type of encoding will be sufficient for modeling purposes.

In [6]:
# first scale all the numerical data

scaler = StandardScaler()
X = df.iloc[:, 1:193]
X[X.columns] = scaler.fit_transform(X[X.columns])

In [7]:
# Encoding the categorical columns

# one hot encoding the 5 categorical columns
# get list of categorical columns
cat_vars = ['cat1', 'cat2', 'cat3', 'cat4', 'cat5']

# dummify each categorical variable, drop the first column,
# and add it to the numerical columns
for i in cat_vars:
  X = X.join(pd.get_dummies(df[i]).iloc[:, 1:], rsuffix=i)


# Getting the target volumn as its own series
y = df.target

# get a sense of whether or not y is imbalanced
print(sum(y) / len(y))

0.009441273867399421


We can see from the above that the training sample is highly imbalanced, with less than 1% of the samples with a target of 1.

In [8]:
# Getting testing and training set

x_train, x_test, y_train, y_test =  train_test_split(X, y,
                                                     test_size=.33,
                                                     random_state = RANDOM_STATE)

In [9]:
train_label_allocation = sum(y_train) / len(y_train)
test_label_allocation = sum(y_test) / len(y_test)

print('Percentage of ones in training set is: ', train_label_allocation)
print('Percentage of ones in testing set is: ', test_label_allocation)

Percentage of ones in training set is:  0.008833736460195604
Percentage of ones in testing set is:  0.01067463706233988


In [10]:
(train_label_allocation - test_label_allocation) / train_label_allocation

-0.2083943312594039

The percentage of the one label in the testing set is roughly 20% off from the percentage of the one label in the training set.  This could potentially be problematic but steps can be taken to mitigate this.

# Building baseline binary classifiers

In [11]:
# Defining a function to easily get scores in a df

score_cols=['Train_acc', 'Train_prec', 'Train_recall', 'Train_fscore', 'Train_auc',
            'Test_acc','Test_prec', 'Test_recall', 'Test_fscore', 'Test_auc',
            'Model', 'Model_version']
all_scores = pd.DataFrame(columns=score_cols)

def get_scores(train_pred, test_pred, y_train, y_test, mod, version):
  scores_train = precision_recall_fscore_support(y_train, train_pred,
                                                 average='binary')
  acc_train = sum(train_pred == y_train) / len(y_train)
  prec_train = scores_train[0]
  recall_train = scores_train[1]
  f_score_train = scores_train[2]
  auc_train = roc_auc_score(y_train, train_pred)

  scores_test = precision_recall_fscore_support(y_test, test_pred,
                                                average='binary')
  acc_test = sum(test_pred == y_test) / len(y_test)
  prec_test = scores_test[0]
  recall_test = scores_test[1]
  f_score_test = scores_test[2]
  auc_test = roc_auc_score(y_test, test_pred)

  scores = [acc_train, prec_train, recall_train, f_score_train, 
            auc_train, acc_test, prec_test, recall_test, f_score_test,
            auc_test, mod, version]

  df_score = pd.DataFrame(np.reshape(scores, newshape=(1,12)),
                          columns=score_cols)
  return df_score

In [12]:
# Looking at benchmark logistic regression

# using a very low value for C and increasing max_inter so the model can converge
lr_bench = LogisticRegression(random_state=RANDOM_STATE, C=.00001, max_iter=1000)
lr_bench.fit(x_train, y_train)

# getting predicitons for LR model
lr_train_pred = lr_bench.predict(x_train)
lr_test_pred = lr_bench.predict(x_test)

In [13]:
# Looking at benchmark logistic regression

# using a defautl catboost classifier
cat_bench = CatBoostClassifier(random_state=RANDOM_STATE)
cat_bench.fit(x_train, y_train)

# getting predicitons for LR model
cat_train_pred = cat_bench.predict(x_train)
cat_test_pred = cat_bench.predict(x_test)

Learning rate set to 0.026952
0:	learn: 0.6216228	total: 183ms	remaining: 3m 2s
1:	learn: 0.5613878	total: 305ms	remaining: 2m 32s
2:	learn: 0.5066779	total: 413ms	remaining: 2m 17s
3:	learn: 0.4601435	total: 538ms	remaining: 2m 14s
4:	learn: 0.4190732	total: 651ms	remaining: 2m 9s
5:	learn: 0.3808890	total: 763ms	remaining: 2m 6s
6:	learn: 0.3466707	total: 879ms	remaining: 2m 4s
7:	learn: 0.3184302	total: 980ms	remaining: 2m 1s
8:	learn: 0.2925518	total: 1.15s	remaining: 2m 6s
9:	learn: 0.2676237	total: 1.31s	remaining: 2m 9s
10:	learn: 0.2466513	total: 1.42s	remaining: 2m 7s
11:	learn: 0.2252152	total: 1.53s	remaining: 2m 6s
12:	learn: 0.2075687	total: 1.64s	remaining: 2m 4s
13:	learn: 0.1928473	total: 1.77s	remaining: 2m 4s
14:	learn: 0.1793594	total: 1.89s	remaining: 2m 4s
15:	learn: 0.1671384	total: 2.01s	remaining: 2m 3s
16:	learn: 0.1565863	total: 2.14s	remaining: 2m 3s
17:	learn: 0.1445894	total: 2.28s	remaining: 2m 4s
18:	learn: 0.1353641	total: 2.42s	remaining: 2m 5s
19:	lear

In [14]:
# Looking at benchmark logistic regression

# using the default xgboost classifier
xgb_bench = XGBClassifier(random_state=RANDOM_STATE)
xgb_bench.fit(x_train, y_train)

# getting predicitons for LR model
xgb_train_pred = xgb_bench.predict(x_train)
xgb_test_pred = xgb_bench.predict(x_test)

In [15]:
all_scores = all_scores.append(get_scores(lr_train_pred, lr_test_pred, y_train, y_test, 'lr', 'benchmark'))
all_scores = all_scores.append(get_scores(cat_train_pred, cat_test_pred, y_train, y_test, 'cat', 'benchmark'))
all_scores = all_scores.append(get_scores(xgb_train_pred, xgb_test_pred, y_train, y_test, 'xgb', 'benchmark'))

all_scores.set_index(['Model_version', 'Model'])

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_acc Train_prec         Train_recall  \
Model_version Model                                                       
benchmark     lr     0.9911662635398044        0.0                  0.0   
              cat    0.9991586917656956        1.0   0.9047619047619048   
              xgb    0.9919024082448207        1.0  0.08333333333333333   

                            Train_fscore           Train_auc  \
Model_version Model                                            
benchmark     lr                     0.0                 0.5   
              cat     0.9500000000000001  0.9523809523809523   
              xgb    0.15384615384615385  0.5416666666666666   

                               Test_acc Test_prec Test_recall Test_fscore  \
Model_version Model                                                         
benchmark     lr     0.9893253629376602       0.0         0.0         0.0   
              cat     0.988471391972673       0.0         0.0         0.0   
              xgb    0.9891118701964133       0.0         0.0         0.0   

                               Test_auc  
Model_version Model                      
benchmark     lr                    0.5  
              cat    0.4995684074233923  
              xgb    0.4998921018558481

It seems like the every benchmark model performs poorly on the test set, though catboost seems to have pretty good results for the training set.  Next we can try hyperparameter tuning on the baseline classifiers to see if they can be improved.

# Hyperparameter tuning on baseline classifiers

In [16]:
# first tune lr hyperparameters

lr_grid_values = {'C': [0.00001, 0.000001, .0000001]}
lr_tuned = GridSearchCV(LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
                        param_grid=lr_grid_values)
lr_tuned.fit(x_train, y_train)
lr_tuned.best_params_

# This is the same regularizer that we used in the baseline model, so the
# results will end up the same, so we will skip this prediction

{'C': 1e-05}

In [20]:
# tuning catboost hyperparameters

cat_grid_values = {'depth': [4,7,10],
                   'learning_rate' : [0.01,0.03],
                   'iterations'    : [20,40,60,80,100]
                 }
cat_tuned = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE),
                         param_grid=cat_grid_values,
                         cv=3 # not using default cv as the catboost training time is already high
                         )
cat_tuned.fit(x_train, y_train)
cat_tuned.best_params_

# getting the predictions
cat_tuned_train_pred = cat_tuned.predict(x_train)
cat_tuned_test_pred = cat_tuned.predict(x_test)

Streaming output truncated to the last 5000 lines.
0:	learn: 0.6636060	total: 47.5ms	remaining: 2.8s
1:	learn: 0.6403104	total: 86.1ms	remaining: 2.5s
2:	learn: 0.6186845	total: 136ms	remaining: 2.58s
3:	learn: 0.5947876	total: 171ms	remaining: 2.39s
4:	learn: 0.5750209	total: 233ms	remaining: 2.56s
5:	learn: 0.5554274	total: 299ms	remaining: 2.69s
6:	learn: 0.5365667	total: 363ms	remaining: 2.75s
7:	learn: 0.5187418	total: 411ms	remaining: 2.67s
8:	learn: 0.5019048	total: 465ms	remaining: 2.63s
9:	learn: 0.4839404	total: 540ms	remaining: 2.7s
10:	learn: 0.4676159	total: 606ms	remaining: 2.7s
11:	learn: 0.4492475	total: 662ms	remaining: 2.65s
12:	learn: 0.4326671	total: 727ms	remaining: 2.63s
13:	learn: 0.4188498	total: 833ms	remaining: 2.74s
14:	learn: 0.4037393	total: 906ms	remaining: 2.72s
15:	learn: 0.3905791	total: 962ms	remaining: 2.64s
16:	learn: 0.3761005	total: 993ms	remaining: 2.51s
17:	learn: 0.3644532	total: 1.04s	remaining: 2.43s
18:	learn: 0.3529801	total: 1.09s	remaining

In [ ]:
# tuning xgbboost hyperparameters

xgb_grid_values = {'depth': [4,7,10],
                   'learning_rate' : [0.01,0.03],
                   'iterations'    : [20,40,60,80,100]
                 }
xgb_tuned = GridSearchCV(XGBClassifier(random_state=RANDOM_STATE),
                         param_grid=xgb_grid_values
                         )
xgb_tuned.fit(x_train, y_train)
xgb_tuned.best_params_

# getting the predictions
xgb_tuned_train_pred = xgb_tuned.predict(x_train)
xgb_tuned_test_pred = xgb_tuned.predict(x_test)

In [ ]:
all_scores = all_scores.append(get_scores(lr_train_pred, lr_test_pred, y_train, y_test, 'lr', 'benchmark'))
all_scores = all_scores.append(get_scores(cat_train_pred, cat_test_pred, y_train, y_test, 'cat', 'benchmark'))
all_scores = all_scores.append(get_scores(xgb_train_pred, xgb_test_pred, y_train, y_test, 'xgb', 'benchmark'))

all_scores.set_index(['Model_version', 'Model'])